# NLP Challenge
The dataset is a collection of audio clips with expressions of the following emotions: *angry, fear, happy, neutral, sad*.  
  
Your task is to train a model to perform *speech emotion recognition*.  
  
You will be provided a training set for your use. You are allowed to include additional data to train your model.  
  
The evaluation data will consist of audio clips spoken in multiple languages.  
*Majority* of the evaluation data are in the intonation of *Singapore English*.  
  
Take inspo from [training notebook](https://github.com/AbishekSankar/Audio-Classification-Deep-Learning/blob/main/Demo%20Jupyter%20Notebook/Final_Project.ipynb)  
and [Speech Emotion Recognition with CNN](https://www.kaggle.com/code/ritzing/speech-emotion-recognition-with-cnn/notebook)

## Possible Extra Datasets
https://www.kaggle.com/datasets/dmitrybabko/speech-emotion-recognition-en (specifically Crema)  
https://www.kaggle.com/datasets/piyushagni5/berlin-database-of-emotional-speech-emodb  

In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

## Audio Data Processing

In [2]:
emotions={
  '1':'angry',
  '2':'fear',
  '3':'happy',
  '4':'neutral',
  '5':'sad'
}

emotions_list = ['angry', 'fear', 'happy', 'neutral', 'sad']

In [3]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    # print(type(result))
    # print(result.shape)
    return result

In [4]:
x_train, y_train = [], []

for emotion in emotions_list:
    filepath = 'Data/NLP Training Dataset/ASR Training Dataset/' + emotion + '/*'
    for file in glob.glob(filepath):
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x_train.append(feature)
        y_train.append(emotion)

x_train = np.array(x_train)
y_train = np.array(y_train)

C:\Users\mandy\AppData\Local\Temp\ipykernel_31284\2108017461.py:16: FutureWarning: Pass y=[-0.00682908 -0.00637656 -0.00541121 ... -0.00074142 -0.00080585
 -0.00092882] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\mandy\AppData\Local\Temp\ipykernel_31284\2108017461.py:16: FutureWarning: Pass y=[ 7.6752901e-04  1.0554194e-03  5.8865547e-04 ...  3.8468838e-04
 -2.1868944e-04  3.9339066e-06] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\mandy\AppData\Local\Temp\ipykernel_31284\2108017461.py:16: FutureWarning: Pass y=[ 0.00170732  0.00163615 -0.00534987 ... -0.00061619 -0.00046349
 -0.00167292] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspe

In [5]:
print(type(x_train))
print(x_train.shape)

<class 'numpy.ndarray'>
(3500, 180)


In [6]:
# shuffle order of data and label to match
def unison_shuffled_arrays(x, y):
    assert len(x) == len(y)
    p = np.random.permutation(len(x))
    return x[p], y[p]

shuffle_x_train, shuffle_y_train = unison_shuffled_arrays(x_train, y_train)

In [7]:
print(shuffle_x_train.shape)

print(len(shuffle_x_train[5]))

(3500, 180)
180


In [13]:
shuffle_x_train[0].shape

(180,)

## Build Model

In [14]:
xIn = Input(shape=(shuffle_x_train[0].shape))
net = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False)
x = net(xIn)
x = Flatten()(x)
x = Dense(128, activation='swish')(x)
x = Dropout(0.5)(x)
xOut = Dense(5, activation='softmax')(x)

model = Model(xIn, xOut)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

ValueError: Exception encountered when calling layer "efficientnetv2-s" (type Functional).

Input 0 of layer "stem_conv" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 180)

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 180), dtype=float32)
  • training=False
  • mask=None

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(r'Model_weights', monitor='val_loss', verbose=1, save_best_only=True)
]

In [ ]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_train)
accuracy=accuracy_score(y_true=y_train, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
x_test = []
filepath = 'Data/NLP Interim Dataset/NLP Interim Dataset/NLP/*'
for file in glob.glob(filepath):
    feature = extract_feature(file, mfcc=False, chroma=False, mel=True)
    x_test.append(feature)

In [ ]:
y_test = model.predict(x_test)
print(y_test)